### Setting up

In [ ]:
# Import packages
import pandas as pd
from os import environ
from tqdm import tqdm
from sys import exit
from time import sleep
from auth import Auth
from workspace import Workspace
from dataset import Dataset
from openpyxl import load_workbook

# Tenant/app settings
TENANT_ID = environ.get('TENANT_ID', '')
CLIENT_ID = environ.get('CLIENT_ID', '')
CLIENT_SECRET = environ.get('CLIENT_SECRET', '')

# Save access clean up file
FILENAME = './data/datasets/datasets_cleanup.xlsx'

In [ ]:
# Authentication (get bearer token)
auth = Auth(TENANT_ID, CLIENT_ID, CLIENT_SECRET)
token = auth.get_token()

# Initializing objects
workspace = Workspace(token)
dataset = Dataset(token)

### Getting workspaces data

In [ ]:
# Term to search on workspace name
# workspace_to_search = 'Brewdat'
# workspaces = workspace.list_workspaces(filters=f"contains(name,'{workspace_to_search}')%20or%20name%20eq%20'Dataflows'")

# Filter workspaces that contain search word or is Dataflows workspace
# Saves to an Excel (.xslx) file
workspaces = workspace.list_workspaces()
workspaces_list = workspaces.get('content', [])

# See content
n = 1
print(f'Found {len(workspaces_list)} workspaces...\nPrinting first {n}:')
pd.DataFrame(workspaces_list).head(n)

### Getting the list of users to be updated

In [ ]:
# Reads the file with the users, workspaces and reports to remove
df = pd.read_excel('./data/access_to_remove_datasets.xlsx', sheet_name='Dash_Users_With_Email', dtype=str)
df['item_id'] = df['Workspace'] + '_' + df['Report2']

# Workspaces and reports we need to change
workspaces_to_change = df['Workspace'].unique()
items_to_change = {
    'workspaces': {},
    'reports': {}
}

# Get the workspaces IDs
for workspace_to_change in workspaces_to_change:
    for workspace_data in workspaces_list:
        if workspace_to_change.upper() == workspace_data['name'].upper():
            workspace_id = workspace_data['id']
            items_to_change['workspaces'][workspace_to_change] = {}
            items_to_change['workspaces'][workspace_to_change]['id'] = workspace_id
            items_to_change['workspaces'][workspace_to_change]['reports'] = workspace.list_reports(workspace_id)['content']
            break

# Add the workspaces IDs to the dataframe
df['workspace_id'] = [items_to_change['workspaces'][workspace]['id'] for workspace in df['Workspace'].values]

# Get a list of unique combinations of workspaces and reports
df_ = df.loc[:,['Workspace', 'Report2', 'item_id']]
df_.drop_duplicates(subset=['item_id'], inplace=True)

# Get the reports IDs and datasets IDs
for row in df_.to_dict('records'):
    workspace_name = row['Workspace']
    report_name = row['Report2']
    reports_list = items_to_change['workspaces'][workspace_name]['reports']

    for report_data in reports_list:
        if report_name.upper() == report_data['name'].upper():

            items_to_change['reports'][report_name] = {}
            items_to_change['reports'][report_name]['id'] = report_data['id']
            items_to_change['reports'][report_name]['dataset_id'] = report_data['datasetId']
            items_to_change['reports'][report_name]['workspace_id'] = report_data['datasetWorkspaceId']
            break

# Add the reports IDs and datasets IDs to the dataframe
df['report_id'] = [items_to_change['reports'][report]['id'] if report in items_to_change['reports'].keys() else 'remove' for report in df['Report2'].values]
df['dataset_id'] = [items_to_change['reports'][report]['dataset_id'] if report in items_to_change['reports'].keys() else 'remove' for report in df['Report2'].values]

In [ ]:
# Create a final dataframe with only reports that still exists,
# and with columns that we are interested in.
final_df = df.loc[df['report_id']!='remove', ['emailAddress', 'reportUserAccessRight', 'workspace_id', 'dataset_id', 'report_id']]
final_df.columns = ['user_principal_name', 'access_right', 'workspace_id', 'dataset_id', 'report_id']

# Create a new empty column to store update status of the request
final_df['update_status'] = ''
final_df.reset_index(drop=True, inplace=True)

# Save to an Excel file
filename = './data/datasets/datasets_cleanup.xlsx'
final_df.to_excel(FILENAME, index=False)
final_df.head(3)

In [ ]:
# Loop for each user to be changed
users_updated = 0
final_df = pd.read_excel(FILENAME, dtype=str)
final_df.fillna('', inplace=True)

SAVE_EVERY_N_ITERATIONS = 10
REQUESTS_PER_HOUR = 180

for index, row in tqdm(enumerate(final_df.to_dict('records'))):
    user = row['user_principal_name']
    workspace_id = row['workspace_id']
    dataset_id = row['dataset_id']
    access_right = row['access_right']
    current_update_status = row['update_status']

    # If not already updated...
    if (current_update_status == '') & (access_right != 'Owner'):

        try:

            # Try to remove the user access to the dataset
            response = dataset.remove_user(
                        user_principal_name=user,
                        workspace_id=workspace_id,
                        dataset_id=dataset_id)

            # Request status
            status = response['message']

            # If success...
            if 'Success' in status:
                update_status = 'Done'

            # Any other error...
            else:
                update_status = status['error']['description']

            # Update the status column on the worksheet
            final_df.loc[index,'update_status'] = update_status

            # Update the number of updated users
            users_updated += 1

            # Save dataframe every 10 requests
            if (index+1) % SAVE_EVERY_N_ITERATIONS == 0:
                final_df.to_excel(filename, index=False)

            # After 180 requests save dataframe,
            # #and wait an hour to avoid getting blocked (HTTP ERROR 429).
            if (users_updated+1) % REQUESTS_PER_HOUR == 0:
                final_df.to_excel(filename, index=False)
                print('\nMaximum requests per hour reached, sleeping for an hour...\n')
                sleep(3660)

                # Reauthenticate
                auth = Auth(TENANT_ID, CLIENT_ID, CLIENT_SECRET)
                token = auth.get_token()
                dataset = Dataset(token)

            sleep(3)

        except Exception as e:
            print('\nError:', e)
            if users_updated > 0:
                final_df.to_excel(filename, index=False)
            exit()

# If there are any updates, save the file
if users_updated > 0:
    final_df.to_excel(filename, index=False)